In [ ]:
# !pip install kaggle
# from google.colab import files
# files.upload()

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Nov  1 10:17:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# !ls -1ha kaggle.json

In [ ]:
!pip install iterative-stratification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 35.0 MB/s 
     |████████████████████████████████| 7.6 MB 89.3 MB/s 
     |████████████████████████████████| 163 kB 77.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 23.8 MB/s 


In [ ]:
!pip3 install tokenizers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 34.9 MB/s 
     |████████████████████████████████| 182 kB 91.2 MB/s 
     |████████████████████████████████| 166 kB 79.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 101.0 MB/s 
     |████████████████████████████████| 162 kB 91.5 MB/s 
     |████████████████████████████████| 162 kB 102.0 MB/s 
     |████████████████████████████████| 158 kB 87.6 MB/s 
     |████████████████████████████████| 157 kB 109.7 MB/s 
     |████████████████████████████████| 157 kB 98.3 MB/s 
     |████████████████████████████████| 157 kB 108.8 MB/s 
     |████████████████████████████████| 157 kB 109.2 MB/s 
     |████████████████████████████████| 157 kB 104.1 MB/s 
     |████████████████████████████████| 157 kB 103.0 MB/s 
     |████████████████████████████████| 157 kB 94.1 MB/s 
     |████████████████████

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# # Permission Warning 이 일어나지 않도록 
# !chmod 600 ~/.kaggle/kaggle.json
# # 본인이 참가한 모든 대회 보기 
# !kaggle competitions list

In [ ]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Kaggle')
os.chdir('Feedback3')

In [ ]:
# !kaggle competitions download -c feedback-prize-english-language-learning

In [ ]:
# !unzip feedback-prize-english-language-learning.zip

# !ls

In [ ]:
OUTPUT_DIR = './fb3-dbv3b-outputs/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

## CFG

In [ ]:
class CFG:
    wandb=True
    competition='FB3'
    _wandb_kernel='bluehills'
    debug=False
    apex=True
    print_freq=20
    model="microsoft/deberta-v3-base" #["microsoft/deberta-v3-large", "microsoft/deberta-large-mnli", "microsoft/deberta-v2-xlarge", "microsoft/deberta-v2-xlarge-mnli"]
    gradient_checkpointing=True
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=50
    epochs=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=16
    max_len=512
    weight_decay=0.01
    weight_decay_decoder=0.00
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    train=True

    # CPMP
    num_workers=1
    sync_bn = False


if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
# ====================================================
# wandb
# ====================================================
if CFG.wandb:
    import wandb
    try:
        # from kaggle_secrets import UserSecretsClient
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("wandb_api")
        # wandb.login(key=secret_value_0)
        print('login to wandb')
        wandb.login()
        anony = None
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')


    def class2dict(f):
        return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

    run = wandb.init(project='Feedback3-Public', 
                     name=CFG.model,
                     config=class2dict(CFG),
                     group=CFG.model,
                     job_type="train",
                     anonymous=anony)

login to wandb


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bluehills. Use `wandb login --relogin` to force relogin


## Library

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import argparse

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import mean_squared_error
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# PyTorch
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

# CPMP imports for DDP
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data.distributed import DistributedSampler


import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.13.1
transformers.__version__: 4.23.1
env: TOKENIZERS_PARALLELISM=true


## Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

## Data Loading

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

print(f"train.shape: {train.shape}")
display(train.head())
print(f"test.shape: {test.shape}")
display(test.head())
print(f"submission.shape: {submission.shape}")
display(submission.head())

train.shape: (3911, 8)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


test.shape: (3, 2)


,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


submission.shape: (3, 7)


,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


## CV split

In [ ]:
# ====================================================
# CV split
# ====================================================
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols])):
    train.loc[val_index, 'fold'] = int(n)

train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    391
1    391
2    391
3    391
4    391
5    391
6    391
7    391
8    391
9    392
dtype: int64

In [ ]:
if CFG.debug:
    display(train.groupby('fold').size())
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train.groupby('fold').size())

## Tokenizer

In [ ]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(OUTPUT_DIR+'tokenizer/')
CFG.tokenizer = tokenizer

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Dataset

In [ ]:
# ====================================================
# Define max_len
# ====================================================
# lengths = []
# tk0 = tqdm(train['full_text'].fillna("").values, total=len(train))
# for text in tk0:
#     length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
#     lengths.append(length)
# CFG.max_len = max(lengths) + 3 # cls & sep & sep
# LOGGER.info(f"max_len: {CFG.max_len}")

CFG.max_len = 1024

In [ ]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs, label
    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

## Model

In [ ]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    


class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            self.config.relative_attention = True
            self.config.max_position_embeddings = cfg.max_len
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config, ignore_mismatched_sizes=True) #torch_dtype=torch.float16
        else:
            self.model = AutoModel(self.config)

        
        # Freeze the hidden layers for CUDA memory issue
        if 'deberta-v2-xxlarge' in cfg.model:
            print('freeze 32 deberta layers')
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:32].requires_grad_(False) # 32/48
        if 'deberta-v2-xlarge' in cfg.model:
            print('freeze 12 deberta layers')
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:12].requires_grad_(False) # 12/24
        if 'deberta-xlarge' in cfg.model:
            print(f'# of total layers = {len(self.model.encoder.layer)}')
            print('freeze 30 deberta layers')
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:30].requires_grad_(False) # 30/48
        if 'funnel-transformer-xlarge' in cfg.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.blocks[:1].requires_grad_(False) # 1/3


        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        
        # attention-based stacked module
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        # pooling layer for the concatenated tensors
        self.concat_pool = nn.Linear(self.config.hidden_size*3, self.config.hidden_size)

        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 6)
        self._init_weights(self.fc)
        
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        mean_feature = self.pool(last_hidden_states, inputs['attention_mask'])

        # attention based sentence representation
        weights = self.attention(last_hidden_states)
        attention_feature = torch.sum(weights * last_hidden_states, dim=1)

        cls_token_feature = last_hidden_states[:, 0, :] # only cls token

        # combine mean pooled feature, attention feature, and cls token feature to the stacked feature
        combine_feature = torch.cat([mean_feature, attention_feature, cls_token_feature], dim = -1)
        concat_pooled_feature = self.concat_pool(combine_feature)
        
        return concat_pooled_feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

In [ ]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
            self.backup = {}

In [ ]:
class AWP:
    def __init__(self, model, optimizer, adv_param='weight', adv_lr=0.001, adv_eps=0.001):
        self.model = model
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.backup = {}

    def perturb(self, input_ids, attention_mask, y, criterion):
        """
        Perturb model parameters for AWP gradient
        Call before loss and loss.backward()
        """
        self._save()  # save model parameters
        self._attack_step()  # perturb weights

    def _attack_step(self):
        e = 1e-6
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                grad = self.optimizer.state[param]['exp_avg']
                norm_grad = torch.norm(grad)
                norm_data = torch.norm(param.detach())

                if norm_grad != 0 and not torch.isnan(norm_grad):
                    # Set lower and upper limit in change
                    limit_eps = self.adv_eps * param.detach().abs()
                    param_min = param.data - limit_eps
                    param_max = param.data + limit_eps

                    # Perturb along gradient
                    # w += (adv_lr * |w| / |grad|) * grad
                    param.data.add_(grad, alpha=(self.adv_lr * (norm_data + e) / (norm_grad + e)))

                    # Apply the limit to the change
                    param.data.clamp_(param_min, param_max)

    def _save(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.clone().detach()
                else:
                    self.backup[name].copy_(param.data)

    def restore(self):
        """
        Restore model parameter to correct position; AWP do not perturbe weights, it perturb gradients
        Call after loss.backward(), before optimizer.step()
        """
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data.copy_(self.backup[name])

In [ ]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

## Helper function

In [ ]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


# CPMP utility to gather values from all workers
def my_gather(output, local_rank, world_size):
    # output must be a tensor on the cuda device
    # output must have the same size in all workers
    result = None
    if local_rank == 0:
        result = [torch.empty_like(output) for _ in range(world_size)]
    torch.distributed.gather(output, gather_list=result, dst=0)
    return result


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler,  device, local_rank, world_size):
    # CPMP pass rank, size as arg

    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0

    fgm = FGM(model)
    awp = AWP(model, optimizer, adv_param="weight", adv_lr=1e-5, adv_eps=1e-6)

    awp_start = 10.0
    fgm_start = 1.0

    for step, (inputs, labels) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        #======================
        # AWP attack
        if epoch >= awp_start:
            awp.perturb(inputs['input_ids'], inputs['attention_mask'], labels, criterion)
        #======================

        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        # loss.backward()

        #======================
        # AWP restore
        if epoch >= awp_start:
            awp.restore()
        #======================

        #======================
        # FGM
        #======================
        if epoch >= fgm_start:
            fgm.attack()
            with torch.cuda.amp.autocast(enabled=CFG.apex):
                y_preds = model(inputs)
                loss_adv = criterion(y_preds, labels)
            scaler.scale(loss_adv).backward()
            # loss_adv.backward()
            fgm.restore()
        #======================

        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            # optimizer.step()
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
        if CFG.wandb:
            wandb.log({f"[fold{fold}] loss": losses.val,
                       f"[fold{fold}] lr": scheduler.get_lr()[0]})
        
        torch.cuda.empty_cache()
        gc.collect()
        
    torch.cuda.empty_cache()
    gc.collect()

    # CPMP gather loss on worker 0
    loss_avg = torch.tensor([losses.avg], device=device)
    loss_avg = my_gather(loss_avg, local_rank, world_size)
    if local_rank == 0:
        loss_avg = torch.cat(loss_avg).mean().item()
    else:
        loss_avg = None
    return loss_avg


def valid_fn(valid_loader, model, criterion, device, local_rank, world_size, valid_dataset):
    # CPMP pass rank, size dataset as arg)
    
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if local_rank == 0: # CPMP
            if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
                print('EVAL: [{0}/{1}] '
                      'Elapsed {remain:s} '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      .format(step, len(valid_loader),
                              loss=losses,
                              remain=timeSince(start, float(step+1)/len(valid_loader))))
        torch.cuda.empty_cache()
        gc.collect()
    predictions = torch.cat(preds) # CPMP
    
    # CPMP gather on worker 0
    loss_avg = torch.tensor([losses.avg], device=device)
    loss_avg = my_gather(loss_avg, local_rank, world_size)
    predictions = my_gather(predictions, local_rank, world_size)
    
    # CPMP gather on worker 0
    if local_rank == 0:
        loss_avg = torch.cat(loss_avg).mean().item()
        predictions = torch.stack(predictions)
        # DistributedSampler interleaves workers
        # transpose restores the original order
        _, _, t = predictions.shape
        predictions = predictions.transpose(0, 1).reshape(-1, t) 
        # DistributedSampler pads the dataset to get a multiple of world size
        predictions = predictions[:len(valid_dataset)]
        predictions = predictions.cpu().numpy()
        return loss_avg, predictions
    else:
        return None, None

In [ ]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold, device, local_rank, world_size):  # CPMP pass device, rank, size as arg
    
    if local_rank == 0: # CPMP
        LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_sampler = DistributedSampler(train_dataset,
                                       num_replicas=world_size,
                                       rank=local_rank,
                                       shuffle=True,
                                       seed=42,
                                       drop_last=True,
                                       )
    valid_sampler = DistributedSampler(valid_dataset,
                                       num_replicas=world_size,
                                       rank=local_rank,
                                       shuffle=False,
                                       seed=42,
                                       drop_last=False,
                                       )

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False, # CPMP
                              sampler=train_sampler, # CPMP
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False, # CPMP
                              sampler=valid_sampler, # CPMP
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR+'config.pth')
    model.to(device)

    # wrap for DDP
    if CFG.sync_bn:
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
    model = DistributedDataParallel(model, device_ids=[local_rank])
    
    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0, weight_decay_decoder=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        # CPMP: when using DDP the original model is now the module attribute
        optimizer_parameters = [
            {'params': [p for n, p in model.module.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.module.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.module.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay,
                                                weight_decay_decoder=CFG.weight_decay_decoder)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)


    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.SmoothL1Loss(reduction='mean')
    # criterion = RMSELoss(reduction="mean")
    
    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        # CPMP pass rank and world size
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, 
                            device, local_rank, world_size) 

        # eval
        # CPMP pass rank, world size, and valid dataset
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, 
                                             device, local_rank, world_size, valid_dataset) 
        
        # scoring
        if local_rank == 0: # CPMP
            score, scores = get_score(valid_labels, predictions)

            elapsed = time.time() - start_time

            LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
            LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
            if CFG.wandb:
                wandb.log({f"[fold{fold}] epoch": epoch+1, 
                           f"[fold{fold}] avg_train_loss": avg_loss, 
                           f"[fold{fold}] avg_val_loss": avg_val_loss,
                           f"[fold{fold}] score": score})

            if best_score > score:
                best_score = score
                LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
                # CPMP: save the original model. It is stored as the module attribute of the DP model.
                torch.save({'model': model.module.state_dict(),
                            'predictions': predictions},
                            OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    if local_rank == 0: # CPMP
        predictions = torch.load(OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
        valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

        torch.cuda.empty_cache()
        gc.collect()

        return valid_folds
    else:
        return None

## Training

In [ ]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[CFG.target_cols].values
        preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    

    torch.distributed.init_process_group(backend="nccl", )
    local_rank = int(os.environ["LOCAL_RANK"])
    world_size = torch.distributed.get_world_size()
    device = "cuda:%s" % local_rank


    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold, device, local_rank, world_size) # CPMP
                if local_rank == 0: # CPMP
                    oof_df = pd.concat([oof_df, _oof_df])
                    LOGGER.info(f"========== fold: {fold} result ==========")
                    get_result(_oof_df)
        if local_rank == 0: # CPMP
            oof_df = oof_df.reset_index(drop=True)
            LOGGER.info(f"========== CV ==========")
            get_result(oof_df)
            oof_df.to_pickle(OUTPUT_DIR+'oof_df.pkl')
        
    if CFG.wandb and local_rank == 0: # CPMP
        wandb.finish()

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 1024,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.23.1",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

INFO:__main__:Debert

Downloading:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: [1][0/220] Elapsed 0m 6s (remain 25m 13s) Loss: 2.6307(2.6307) Grad: 687010.1875  LR: 0.00000040  
Epoch: [1][20/220] Elapsed 1m 18s (remain 12m 24s) Loss: 1.9731(2.5571) Grad: 725475.5625  LR: 0.00000840  
Epoch: [1][40/220] Elapsed 2m 31s (remain 10m 59s) Loss: 0.9498(2.1719) Grad: 803341.9375  LR: 0.00001640  
Epoch: [1][60/220] Elapsed 3m 48s (remain 9m 54s) Loss: 0.1468(1.5812) Grad: 250520.6562  LR: 0.00001999  
Epoch: [1][80/220] Elapsed 5m 5s (remain 8m 44s) Loss: 0.1115(1.2353) Grad: 182405.9219  LR: 0.00001996  
Epoch: [1][100/220] Elapsed 6m 19s (remain 7m 27s) Loss: 0.1034(1.0164) Grad: 177441.4375  LR: 0.00001988  
Epoch: [1][120/220] Elapsed 7m 32s (remain 6m 9s) Loss: 0.1101(0.8702) Grad: 200393.6094  LR: 0.00001978  
Epoch: [1][140/220] Elapsed 8m 52s (remain 4m 58s) Loss: 0.1026(0.7644) Grad: 264691.8750  LR: 0.00001963  
Epoch: [1][160/220] Elapsed 10m 1s (remain 3m 40s) Loss: 0.1129(0.6840) Grad: 136949.9688  LR: 0.00001945  
Epoch: [1][180/220] Elapsed 11m 15

Epoch 1 - avg_train_loss: 0.5326  avg_val_loss: 0.1088  time: 863s
INFO:__main__:Epoch 1 - avg_train_loss: 0.5326  avg_val_loss: 0.1088  time: 863s
Epoch 1 - Score: 0.4666  Scores: [0.5089415376609027, 0.4495655473407141, 0.4331213180053301, 0.4252158696444778, 0.5219629489156603, 0.4609638273360511]
INFO:__main__:Epoch 1 - Score: 0.4666  Scores: [0.5089415376609027, 0.4495655473407141, 0.4331213180053301, 0.4252158696444778, 0.5219629489156603, 0.4609638273360511]
Epoch 1 - Save Best Score: 0.4666 Model
INFO:__main__:Epoch 1 - Save Best Score: 0.4666 Model


...

Epoch: [4][0/219] Elapsed 0m 9s (remain 33m 30s) Loss: 0.1079(0.1079) Grad: 190364.8906  LR: 0.00000753  
Epoch: [4][20/219] Elapsed 2m 22s (remain 22m 20s) Loss: 0.0934(0.0950) Grad: 235415.6094  LR: 0.00000695  
Epoch: [4][40/219] Elapsed 4m 43s (remain 20m 32s) Loss: 0.0806(0.0929) Grad: 175861.9844  LR: 0.00000639  
Epoch: [4][60/219] Elapsed 7m 11s (remain 18m 38s) Loss: 0.1213(0.0945) Grad: 534641.2500  LR: 0.00000583  
Epoch: [4][80/219] Elapsed 9m 39s (remain 16m 27s) Loss: 0.1183(0.0963) Grad: 307228.2500  LR: 0.00000530  
Epoch: [4][100/219] Elapsed 12m 6s (remain 14m 8s) Loss: 0.0993(0.0953) Grad: 213015.0312  LR: 0.00000478  
Epoch: [4][120/219] Elapsed 14m 26s (remain 11m 41s) Loss: 0.1136(0.0974) Grad: 307700.7500  LR: 0.00000428  
Epoch: [4][140/219] Elapsed 16m 55s (remain 9m 21s) Loss: 0.1156(0.0973) Grad: 371591.3125  LR: 0.00000380  
Epoch: [4][160/219] Elapsed 19m 14s (remain 6m 55s) Loss: 0.0551(0.0967) Grad: 311666.0000  LR: 0.00000334  
Epoch: [4][180/219] Elapse

Epoch 4 - avg_train_loss: 0.0960  avg_val_loss: 0.1058  time: 1624s
INFO:__main__:Epoch 4 - avg_train_loss: 0.0960  avg_val_loss: 0.1058  time: 1624s
Epoch 4 - Score: 0.4597  Scores: [0.5173550788874349, 0.4471438625274376, 0.40727531187181465, 0.46087489594109093, 0.5052399263709766, 0.42016884947872196]
INFO:__main__:Epoch 4 - Score: 0.4597  Scores: [0.5173550788874349, 0.4471438625274376, 0.40727531187181465, 0.46087489594109093, 0.5052399263709766, 0.42016884947872196]
Epoch 4 - Save Best Score: 0.4597 Model
INFO:__main__:Epoch 4 - Save Best Score: 0.4597 Model


Epoch: [5][0/219] Elapsed 0m 6s (remain 23m 21s) Loss: 0.0811(0.0811) Grad: 276018.5625  LR: 0.00000213  
Epoch: [5][20/219] Elapsed 2m 22s (remain 22m 28s) Loss: 0.0791(0.0869) Grad: 318587.7188  LR: 0.00000177  
Epoch: [5][40/219] Elapsed 4m 43s (remain 20m 31s) Loss: 0.0754(0.0907) Grad: 183783.1875  LR: 0.00000145  
Epoch: [5][60/219] Elapsed 7m 8s (remain 18m 28s) Loss: 0.0555(0.0905) Grad: 272922.2500  LR: 0.00000115  
Epoch: [5][80/219] Elapsed 9m 46s (remain 16m 38s) Loss: 0.0921(0.0916) Grad: 331408.9688  LR: 0.00000089  
Epoch: [5][100/219] Elapsed 12m 17s (remain 14m 21s) Loss: 0.0663(0.0914) Grad: 169164.6406  LR: 0.00000066  
Epoch: [5][120/219] Elapsed 14m 36s (remain 11m 50s) Loss: 0.0879(0.0920) Grad: 316610.7188  LR: 0.00000046  
Epoch: [5][140/219] Elapsed 17m 3s (remain 9m 26s) Loss: 0.1160(0.0917) Grad: 470716.4688  LR: 0.00000030  
Epoch: [5][160/219] Elapsed 19m 23s (remain 6m 59s) Loss: 0.1080(0.0925) Grad: 195222.1719  LR: 0.00000017  
Epoch: [5][180/219] Elapse

Epoch 5 - avg_train_loss: 0.0926  avg_val_loss: 0.1049  time: 1645s
INFO:__main__:Epoch 5 - avg_train_loss: 0.0926  avg_val_loss: 0.1049  time: 1645s
Epoch 5 - Score: 0.4578  Scores: [0.5160498812833167, 0.4403666973715476, 0.407948479277586, 0.46074765153943603, 0.503319115464221, 0.41824758806450624]
INFO:__main__:Epoch 5 - Score: 0.4578  Scores: [0.5160498812833167, 0.4403666973715476, 0.407948479277586, 0.46074765153943603, 0.503319115464221, 0.41824758806450624]
Epoch 5 - Save Best Score: 0.4578 Model
INFO:__main__:Epoch 5 - Save Best Score: 0.4578 Model
========== fold: 9 result ==========
INFO:__main__:========== fold: 9 result ==========
Score: 0.4578  Scores: [0.5160498812833167, 0.4403666973715476, 0.407948479277586, 0.46074765153943603, 0.503319115464221, 0.41824758806450624]
INFO:__main__:Score: 0.4578  Scores: [0.5160498812833167, 0.4403666973715476, 0.407948479277586, 0.46074765153943603, 0.503319115464221, 0.41824758806450624]
========== CV ==========
INFO:__main__:=====

[fold0] avg_train_loss,█▁▁▁▁
[fold0] avg_val_loss,█▅▃▄▁
[fold0] epoch,▁▃▅▆█
[fold0] loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold0] lr,▃▅███████▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
[fold0] score,█▆▃▄▁
[fold1] avg_train_loss,█▁▁▁▁
[fold1] avg_val_loss,█▃▂▂▁
[fold1] epoch,▁▃▅▆█
[fold1] loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
[fold1] lr,▃▅███████▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
